# Deploying a forcasting model as an API with FastAPI

<a href="https://colab.research.google.com/drive/1uu8TMTIFmDeoFzLtKRfKUIVTUut4X5ga" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab">
</a>


Return to the [castle](https://github.com/Nkluge-correa/TeenyTinyCastle).

Deploying ML models as APIs is an essential component of `MLOps` (Machine Learning Operations). `MLOps` is the practice of managing and automating the entire lifecycle of a machine learning model, including development, deployment, monitoring, and maintenance.

Deploying machine learning models as APIs enables data scientists and engineers to share their work with other members of the team or the outside world, allowing them to use the model's capabilities in their own applications easily.

In this notebook, we will request an API that can be launched from the `xgboost_api.py` file in it. We created an API using `FastAPI` and `Uvicorn`.

`FastAPI` is a modern, fast (high-performance) web framework for building APIs with Python. It is built on top of Starlette for the web parts and Pydantic for the data parts. `FastAPI` allows you to quickly build APIs using asynchronous operations, automatically validating request parameters, and data serialization, among other features.

`Uvicorn` is a lightning-fast `ASGI` (_Asynchronous Server Gateway Interface_) server allowing asynchronous processing in web applications built with frameworks like `FastAPI`. It is built on top of the `asyncio` framework and provides an easy-to-use interface for running ASGI applications in production.

<img src="https://fastapi.tiangolo.com/img/logo-margin/logo-teal.png" alt="drawing" width="400"/>

[Source](https://fastapi.tiangolo.com/).

We are using the same `xgboost` model we explored in [this notebook](https://github.com/Nkluge-correa/TeenyTinyCastle/blob/master/ML-Intro-Course/14_time_series_forecasting.ipynb). The [`xgboost_api.py`](https://github.com/Nkluge-correa/TeenyTinyCastle/blob/master/ML-Intro-Course/20_xgboost_api.py) file is basically an implementation of that notebook as an API that takes a series of dates and sales, trains an `XGBoost` model, and returns the predictions for the future by a fixed amount (plus some statistical information about your data distribution).

Below, we are just making an HTTP request to this API, which is running in parallel with this notebook.

> **Note**: If you are running this notebook on your machine, you have to run the [`xgboost_api.py`](https://github.com/Nkluge-correa/TeenyTinyCastle/blob/master/ML-Intro-Course/20_xgboost_api.py) file before, to create an API instance on your local device. If you are running this on Colab, you can send a request to the live version of this API, which is hosted (for free!) on Render. It can take a while for you to receive a reply (it is free hosting ...), but you should get a JSON response with predictions.


In [1]:
!pip install datasets -q

from datasets import load_dataset

# load the datasets from the hub
df = load_dataset('AIRES-PUCRS/time_series_data')

# turn the datasets into a pandas.DataFrame
df = df['train'].to_pandas()

display(df.head())

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1098 [00:00<?, ? examples/s]

,dates,product_id,sales
0,2020-01-01,chocolate,137.0
1,2020-01-02,chocolate,87.0
2,2020-01-03,chocolate,188.0
3,2020-01-04,chocolate,286.0
4,2020-01-05,chocolate,156.0


If you are trying to use the `https://teeny-tiny-api.onrender.com/predict`, go to the api [Home page](https://teeny-tiny-api.onrender.com/) to awake the endpoint before doing a request. Or, simply run the API locally.

> **Note: This is a simple model and should be used for something other than real forecasting applications.**

In [3]:
import pprint
import requests
import pandas as pd

# The API endpoint URL (you must be running the API locally, a.k.a. run the `20_xgboost_api.py` file)
url = "http://127.0.0.1:8000/predict" # or https://teeny-tiny-api.onrender.com/predict

# Define the input data
data = {
    "product": df.product_id[0], # name of the product
    "dates": list(df.dates), # the list of dates
    "sales": list(df.sales), # the list of sales
    "ahead": 15 # how many days ahead we want to look
}

# Send a POST request with the input data
response = requests.post(url, json=data)

pprint.pprint(response.json())

{'chocolate': {'dates': ['2023-01-03',
                         '2023-01-04',
                         '2023-01-05',
                         '2023-01-06',
                         '2023-01-07',
                         '2023-01-08',
                         '2023-01-09',
                         '2023-01-10',
                         '2023-01-11',
                         '2023-01-12',
                         '2023-01-13',
                         '2023-01-14',
                         '2023-01-15',
                         '2023-01-16',
                         '2023-01-17'],
               'sales': [121.48497772216797,
                         107.69721221923828,
                         107.03388214111328,
                         114.45438385009766,
                         120.33972930908203,
                         124.79177856445312,
                         108.23922729492188,
                         130.49281311035156,
                         105.45321655273438,
         

And it is done. We have our predictions for the future. In theory, we could send any sales history to this API, and it would generate the forecast for us. Let us see our results. 🙃



In [4]:
from IPython.display import Markdown

results_df = pd.DataFrame({
    'dates': response.json()[next(iter(response.json()))]['dates'],
    'sales': response.json()[next(iter(response.json()))]['sales']})

display(Markdown(f"""# `{next(iter(response.json()))}` Report\n{"-"*50}

{results_df.set_index('dates').to_markdown()}

**Total Sales for the next 15 days: {results_df.sales.sum():.2f} Kg.**

## Statistics Report

- **`Mean`:** {response.json()[next(iter(response.json()))]['statistics']['mean']:.2f} Kg.
- **`Minimum`:** {response.json()[next(iter(response.json()))]['statistics']['minimum']:.2f} Kg.
- **`Maximum`:** {response.json()[next(iter(response.json()))]['statistics']['maximum']:.2f} Kg.
- **`Variance`:** {response.json()[next(iter(response.json()))]['statistics']['variance']:.2f}.
- **`Standard Deviation`:** {response.json()[next(iter(response.json()))]['statistics']['std']:.2f}.

"""))


# `chocolate` Report
--------------------------------------------------

| dates      |   sales |
|:-----------|--------:|
| 2023-01-03 | 121.485 |
| 2023-01-04 | 107.697 |
| 2023-01-05 | 107.034 |
| 2023-01-06 | 114.454 |
| 2023-01-07 | 120.34  |
| 2023-01-08 | 124.792 |
| 2023-01-09 | 108.239 |
| 2023-01-10 | 130.493 |
| 2023-01-11 | 105.453 |
| 2023-01-12 | 111.34  |
| 2023-01-13 | 110.751 |
| 2023-01-14 | 119.185 |
| 2023-01-15 | 126.856 |
| 2023-01-16 | 126.497 |
| 2023-01-17 | 122.293 |

**Total Sales for the next 15 days: 1756.91 Kg.**

## Statistics Report

- **`Mean`:** 103.35 Kg.
- **`Minimum`:** 0.00 Kg.
- **`Maximum`:** 466.00 Kg.
- **`Variance`:** 2976.16.
- **`Standard Deviation`:** 54.55.



Now, you could deploy this model as a service, using, for example, a Platform as a Service (`PaaS`) to host your `API` (like [`Render`](https://render.com/) or [`Heroku`](https://www.heroku.com)). You could also deploy already trained ML models with more complicated architectures or services. The general blueprint will be the same. However, keeping your model and `API` secure and up-to-date is one of his constant works in `MLOps`.

> Note: Hugging Face also allows you to serve many types of ML applications as an inference end point through their [Inference Endpoints](https://ui.endpoints.huggingface.co/welcome) service.

---

Return to the [castle](https://github.com/Nkluge-correa/TeenyTinyCastle).